<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*



**Solutions mises en oeuvre**

Les solutions de suggestion de tags présentés ici se basent sur des modèles supervisés de machine learning.

Les modèmes mis en oeuvre : 
    * Naive Multinomial Baysien

Ce notebook utilise les données issues des notebooks : 


**P6_DadaAnalysis.ipynb**

**P6_UnsupervizedMethods.ipynb**



In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# 4. Supervized methods

**Loading TF-IDF operator and CSR matrix**

In [ ]:
import p5_util
file_name="./data/csr_matrix_tdif_ngram_2_2.dump"
csr_matrix = p5_util.object_load(file_name)
print(csr_matrix.shape)

file_name="./data/vectorizer_tdif_ngram_2_2.dump"
vectorizer = p5_util.object_load(file_name)

## 4.1. Test dataset vectorization

In [ ]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_test=pd.read_csv(file_name)
print(df_sof_test.shape)

**Test dataset standardization : applies on `Body` column**

In [ ]:
df_sof_test.columns

In [ ]:
import p6_util
ser_sof_body_test = p6_util.p6_df_standardization(df_sof_test.Body)

In [ ]:
csr_matrix_tdif_ngram_2_2_test = vectorizer.transform(ser_sof_body_test)

In [ ]:
csr_matrix_tdif_ngram_2_2_test.A.shape, df_sof_test.shape

**Save of test CSR matrix**

In [ ]:
file_name="./data/csr_matrix_tdif_ngram_2_2_test.dump"
p5_util.object_dump(csr_matrix_tdif_ngram_2_2_test,file_name)

## 4.2. Target processing : list of TAGS are encoded

### 4.2.1. Target : TAGs from Stack Over Flow are loaded

### 4.2.2. Target : TAGs for train dataset are loaded and formated

In [ ]:
import pandas as pd
file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_train_tags=pd.read_csv(file_name).Tags.copy()
ser_sof_train_tags.shape

**Series is resized to fit with Test dataset**

**Tags from train dataset are converted into a lists of words.**

In [ ]:
import p6_util

csr_matrix_encoded_tag = p6_util.p6_encode_ser_tag_2_csrmatrix(ser_sof_train_tags, leading_marker='<', trailing_marker='>')

In [ ]:
csr_matrix_encoded_tag.shape

In [ ]:
import pandas as pd
file_name="./data/QueryResults.csv"
df_sof_tags=pd.read_csv(file_name)
df_sof_tags.shape

In [ ]:
df_sof_tags = df_sof_tags[df_sof_tags.Count>200]
df_sof_tags.shape

In [ ]:
import p6_util

#---------------------------------------------------------------------------
# Markers '<' and '>' are removed from tags.
#---------------------------------------------------------------------------
ser_tag = ser_sof_train_tags.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>')

In [ ]:
df_sof_tags.shape

In [ ]:
import p6_util

list_list_encoded_row = p6_util.p6_encode_target(df_sof_tags.TagName.tolist(), ser_tag.tolist())

In [ ]:
csr_matrix_encoded_tag=sparse.csr_matrix(np.array(list_list_encoded_row))

**Save of encoded train-dataset target**

In [ ]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
p5_util.object_dump(csr_matrix_encoded_train_tags,file_name)
print(csr_matrix_encoded_train_tags.shape)

## 4.3. Applying Naive Bayes classifier

### 4.3.1. Training classifier

**Loading TF-IDF data-set containing trained vectorized questions**

In [ ]:
import p5_util

file_name="./data/csr_matrix_tdif_ngram_2_2.dump"
csr_matrix_tdif_ngram_2_2 = p5_util.object_load(file_name)
print(csr_matrix_tdif_ngram_2_2.shape)

**Loading target train data-set containing encoded TAGs**

In [ ]:
import p5_util

file_name="./data/csr_matrix_encoded_train_tags.dump"
csr_matrix_encoded_train_target = p5_util.object_load(file_name)
print(csr_matrix_encoded_train_target.shape)

**Training classifier**

one versus Rest leads having one classifier per class.

`OneVsRestClassifier` classifier is used because of multiple classes. 

Then, one classifier per class is fitted.



In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

model = OneVsRestClassifier(MultinomialNB(), n_jobs=4).fit(csr_matrix_tdif_ngram_2_2, csr_matrix_encoded_train_target)


### 4.3.2.Classifier evaluation

**Loading test data-set**

In [ ]:
file_name="./data/csr_matrix_tdif_ngram_2_2_test.dump"
csr_matrix_tdif_ngram_2_2_test = p5_util.object_load(file_name)
csr_matrix_tdif_ngram_2_2_test.shape

In [ ]:
model

**Predictions and probabilities**

In [ ]:
csr_matrix_encoded_predict_tags = model.predict(csr_matrix_tdif_ngram_2_2_test)

In [ ]:
import numpy as np
np.where(csr_matrix_encoded_predict_tags[1000].A!=0)

**<font color=blue>P({Tag_i}|Question)</font>**

In [ ]:
csr_matrix_encoded_predict_proba_tags = model.predict_proba(csr_matrix_tdif_ngram_2_2_test)

In [ ]:
len(csr_matrix_encoded_predict_proba_tags[1000])

In [ ]:
import numpy as np
row=3000
np.where(csr_matrix_encoded_predict_proba_tags[row]!=0)
csr_matrix_encoded_predict_proba_tags[row].min(), csr_matrix_encoded_predict_proba_tags[row].max(),csr_matrix_encoded_predict_proba_tags[1000].mean()
np.sort(csr_matrix_encoded_predict_proba_tags[row])[5150:]

**Compute accuracy**

In [ ]:
df_sof_test.columns
ser_sof_test_tags=df_sof_test['Tags']
ser_sof_test_tags.shape

Tests TAGs are encoed then converted into CSR matrix

In [ ]:
csr_matrix_encoded_test_tags = p6_encode_ser_tag_2_csrmatrix(ser_sof_test_tags)

In [ ]:
csr_matrix_tdif_ngram_2_2_test.shape, csr_matrix_encoded_test_tags.shape,csr_matrix_encoded_predict_tags.shape

In [ ]:
score = model.score(csr_matrix_tdif_ngram_2_2_test ,csr_matrix_encoded_test_tags[:,:5155])
print("Mean score for Multinomial Naive Bayse classifier : "+str(score))